# 3. Tasks and Questions

## 3.1 Convergence and attractors

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [19]:
x1 = np.array([[-1, -1, 1, -1, 1, -1, -1, 1]])
x2 = np.array([[-1, -1, -1, -1, -1, 1, -1, -1]])
x3 = np.array([[-1, 1, 1, -1, -1, 1, -1, 1]])

In [32]:
def create_w(patterns_list):
    # Generate a matrix of weights with the patterns
    size = patterns_list[0].size
    w = np.zeros([size,size])
    for pattern in patterns_list:
        w += pattern.T @ pattern
    return(w)

def update_w(patterns_list, w):
    # update an existing weight matrix
    for pattern in patterns_list:
        w += pattern.T @ pattern
    return(w)

def compile(w, x):
    # compile the result of sign( sum ( wij * xj))
    y = np.sign(x @ w)
    return(y)

In [27]:
w = create_w([x1, x2, x3])

In [38]:
# test recall
print('Recall on x1, difference between compilation result and x1')
print(compile(w, x1)-x1)
print('Recall on x2, difference between compilation result and x2')
print(compile(w, x2)-x2)
print('Recall on x2, difference between compilation result and x3')
print(compile(w, x3)-x3)

Recall on x1, difference between compilation result and x1
[[0. 0. 0. 0. 0. 0. 0. 0.]]
Recall on x2, difference between compilation result and x2
[[0. 0. 0. 0. 0. 0. 0. 0.]]
Recall on x2, difference between compilation result and x3
[[0. 0. 0. 0. 0. 0. 0. 0.]]


In [39]:
x1d = np.array([[1, -1, 1, -1, 1, -1, -1, 1]])
x2d = np.array([[1, 1, -1, -1, -1, 1, -1, -1]])
x3d = np.array([[1, 1, 1, -1, 1, 1, -1, 1]])

In [49]:
# Applying update rule until convergence:

# x1d
vect = np.array([i for i in x1d]) # deep copy
diff = 1
i = 0
while diff != 0:
    i += 1
    old_vect = np.array([i for i in vect])
    vect = compile(w, vect)
    diff = np.sum(abs(vect-old_vect))
print("Fixed point for x1d reached in", i, "steps")
print('difference between convergence vect and x1',vect-x1)

# x2d
vect = np.array([i for i in x2d]) # deep copy
diff = 1
i = 0
while diff != 0:
    i += 1
    old_vect = np.array([i for i in vect])
    vect = compile(w, vect)
    diff = np.sum(abs(vect-old_vect))
print("Fixed point for x2d reached in", i, "steps")
print('difference between convergence vect and x2',vect-x2)

# x3d
vect = np.array([i for i in x3d]) # deep copy
diff = 1
i = 0
while diff != 0:
    i += 1
    old_vect = np.array([i for i in vect])
    vect = compile(w, vect)
    diff = np.sum(abs(vect-old_vect))
print("Fixed point for x3d reached in", i, "steps")
print('difference between convergence vect and x3',vect-x3)

Fixed point for x1d reached in 2 steps
difference between convergence vect and x1 [[0. 0. 0. 0. 0. 0. 0. 0.]]
Fixed point for x2d reached in 4 steps
difference between convergence vect and x2 [[0. 0. 0. 0. 0. 0. 0. 0.]]
Fixed point for x3d reached in 3 steps
difference between convergence vect and x3 [[0. 0. 0. 0. 0. 0. 0. 0.]]


In [110]:
# Finding the attractors: We will test for each vector possible what is its state of convergence
# Generating all combination:
combinations = np.array([i for i in product([-1, 1], repeat = 8)])
convergence_vectors = x1
for comb in range(len(combinations)):
    vect = np.array([i for i in combinations[comb]]) # deep copy
    diff = 1
    while diff != 0:
        old_vect = np.array([i for i in vect])
        vect = compile(w, vect)
        diff = np.sum(abs(vect-old_vect))
        # We need to test if vect is in conv but also if -vect is in conv
    vect_in_conv = np.any(np.sum(vect == np.array(convergence_vectors), axis = 1)==8)
    minus_vect_in_conv = np.any(np.sum(-vect == np.array(convergence_vectors), axis = 1)==8)
    if(vect_in_conv ==  False and minus_vect_in_conv == False):
        convergence_vectors = np.concatenate([convergence_vectors, np.array([vect])])
print(convergence_vectors)

[[-1. -1.  1. -1.  1. -1. -1.  1.]
 [-1. -1. -1. -1. -1.  1. -1. -1.]
 [-1. -1.  1. -1. -1.  1. -1.  1.]
 [-1. -1.  0. -1. -1.  1. -1.  0.]
 [-1. -1. -1. -1.  1. -1. -1. -1.]
 [ 1. -1. -1.  1.  1. -1.  1. -1.]
 [ 0.  0. -1.  0. -1.  1.  0. -1.]
 [-1. -1.  1. -1.  0.  0. -1.  1.]]


We have 8 vectors, which is normal because the dimension is 8

As we have seen by testing all the combinations possible, it always converge, whatever the distance we are from the patterns